# Interactive jupyter / IPython notebook Tutorial

This tutorial works on jupyter notebook. Please download examples from here: https://github.com/AB-CE/examples and load the jupyter_tutorial by changing in the `examples/jupyter_tutorial` folder and typing `jupyter notebook jupyter_tutorial.ipynb`. Then you can execute the notebook cell by cell and observe ABCE at work.

In [ ]:
import abcEconomics

In [2]:
simulation = abcEconomics.Simulation()

## Creating the Agent
We create a simple agent and intialize it. An agent is an object that has propeties and does things. In python that is a class. We create an agent that has a name, knows the world size and can say his name. 

The agent needs to inherit from abce.Agents. *The `__init__` function should not be overwritten, instead create an `init` function*. The init function will be called during initialisation.
The agents gets two parameters. All agents get the parameters we will specify in the next round. From agent_parameters the agents get only agent_parameters[id], where id is the number of the agent.
Agents have for now only one function. They can say their name.

In [3]:
class Agent(abcEconomics.Agent):
    def init(self, other_parameter, family_name):
        self.family_name = family_name
        self.other_parameter = other_parameter
        print(other_parameter, family_name)
        
    def say(self):
        print("hello I am %s my id %i and my group is '%s', it is the %i round" % (self.family_name, self.id, self.group, self.round))

From the abstract agent, we create four concrete agents.

In [4]:
agents = simulation.build_agents(Agent, 'agent',
                                 other_parameter=30,
                                 agent_parameters=[{'family_name': 'fred'}, 
                                                   {'family_name': 'astaire'}, 
                                                   {'family_name': 'altair'}, 
                                                   {'family_name': 'deurich'}])

30 fred
30 astaire
30 altair
30 deurich


`agents` allows you to call each agents in the agents group. Each agent in this group has a group name, which is 'agent' and an id. Further we give each of the agents a parameter a family_name.
We can now run the simulation and let all agents say their name:

In [5]:
for r in range(5):
    simulation.time = r
    agents.say()

Round0
hello I am fred my id 0 and my group is 'agent', it is the 0 round
hello I am deurich my id 3 and my group is 'agent', it is the 0 round
hello I am astaire my id 1 and my group is 'agent', it is the 0 round
hello I am altair my id 2 and my group is 'agent', it is the 0 round
Round1
hello I am fred my id 0 and my group is 'agent', it is the 1 round
hello I am deurich my id 3 and my group is 'agent', it is the 1 round
hello I am astaire my id 1 and my group is 'agent', it is the 1 round
hello I am altair my id 2 and my group is 'agent', it is the 1 round
Round2
hello I am fred my id 0 and my group is 'agent', it is the 2 round
hello I am deurich my id 3 and my group is 'agent', it is the 2 round
hello I am astaire my id 1 and my group is 'agent', it is the 2 round
hello I am altair my id 2 and my group is 'agent', it is the 2 round
Round3
hello I am fred my id 0 and my group is 'agent', it is the 3 round
hello I am deurich my id 3 and my group is 'agent', it is the 3 round
hello I

It is necessary to tell the simulation when a new round starts. The parameter can be any representation of time.

## Giving a Good
ABCE provide goods. Goods are things that can be given, sold or transformed.
We create 5 agents, the first one has a balls the agents pass the ball around.

In [6]:
class Kid(abcEconomics.Agent):
    def init(self, num_kids):
        self.num_kids = num_kids
        if self.id == 0:
            self.create('ball', 1)
        
    def whether_I_have_the_ball(self):
        if self['ball'] > 0:
            print('*', end="", flush=True)
        else:
            print('.', end="", flush=True)
            
    def give_the_ball_to_the_next_kid(self):
        next_kid = (self.id + 1) % self.num_kids  # the id of the next kid, if I am the last the first kid
        if self['ball'] >= 1:
            self.give(('kid', next_kid), good='ball', quantity=1)
            
    
        

self.create, creates an object. self.possession, checks how much of one object an agent has. self.give, gives an object to another agent, specied by its group name and its id.

In [7]:
num_kids = 5

In [8]:
simulation = abcEconomics.Simulation(name='ipythonsimulation', processes=1)

In [9]:
kids = simulation.build_agents(Kid, 'kid', number=num_kids,
                               num_kids=num_kids) 
                            

When agent_parameters is not specified the numer of agents to be created needs to be spezified

In [10]:
for r in range(7):
    simulation.time = r
    kids.whether_I_have_the_ball()
    print()
    kids.give_the_ball_to_the_next_kid()

Round0
*....
Round1
.*...
Round2
...*.
Round3
....*
Round4
..*..
Round5
*....
Round6
.*...


## Trade

In [11]:
from random import randrange

Well in every school yard we have a drug dealer.

In [12]:
class NewKid(abcEconomics.Agent):
    def init(self, num_dealers):
        self.num_dealers = num_dealers
        self.create('money', 100)  # don't we all wish you'd this function in real live?
        
    def buy_drugs(self):
        drug_dealer_id = randrange(self.num_dealers)
        self.make_bid(('drug_dealer', drug_dealer_id), good='drugs', quantity=1, price=10)
    
    def print_possessions(self):
        print('    ' + self.group + str(dict(self.possessions())))
        
    

The new kids, approach a random drug dealer and offer him 10 bucks.

In [13]:
class DrugDealer(abcEconomics.Agent):
    def init(self):
        self.create('drugs', 1)
        
    def sell_to_customers(self):
        for offer in self.get_offers('drugs'):
            if offer.price >= 10 and self['drugs'] > 1:
                self.accept(offer)
    
    def print_possessions(self):
        print('    ' + self.group + str(dict(self.possessions())))

Drug dealer look at all the sell offers they get and decide to sell only to those kids that are willing to give them at least 10 dollars.

In [14]:
num_dealers = 1

In [15]:
simulation = abcEconomics.Simulation(name='school_yard', processes=1)

In [16]:
drug_dealers = simulation.build_agents(DrugDealer, 'drug_dealer', number=num_dealers)

In [17]:
customers = simulation.build_agents(NewKid, 'customer', number=1, 
                                   num_dealers=num_dealers)

builds 1 drug dealer and one customer.

In [18]:
kids = drug_dealers + customers

Groups of agents can be merged to 'super' groups. We will print the amount of drugs and money all kids have for each of the two kids

In [19]:
for r in range(2):
    simulation.time = r
    print('Customer offers 10 dollar:')
    customers.buy_drugs()
    kids.print_possessions()
    print('Drug Dealer accepts or rejects the offer:')
    drug_dealers.sell_to_customers()
    kids.print_possessions()
    print()

Round0
Customer offers 10 dollar:
    drug_dealer{'drugs': 1.0}
    customer{'money': 90.0}
Drug Dealer accepts or rejects the offer:
    drug_dealer{'drugs': 1.0}
    customer{'money': 90.0}

Round1
Customer offers 10 dollar:
    drug_dealer{'drugs': 1.0}
    customer{'money': 80.0}
Drug Dealer accepts or rejects the offer:
    drug_dealer{'drugs': 1.0}
    customer{'money': 80.0}



When looking at round one one can see that after the customer offered 10 dollars, the 10 dollars are not available to him util the deal has either been accepted or rejected. After the drug dealer accepts the offer in the 0 round. The money is transfered to the drug dealer and the drugs to the customer.

In round 1, where the drug dealer runs out of drugs the 10 dollars go back to the customer.


## Lets capture data


There are three ways of capturing data. `aggregate` and `panel` collect data from a specified group at a specified point of time. This has the advantage that there is no logging code in the agent class. `self.log('name', value)` saves a value under a certain name.

In [20]:
from math import sin

class DataDealer(abcEconomics.Agent):
    def init(self):
        self.count = 0
        self.create('money', 0)

    def counting(self):
        self.count += 1
        self.curve = sin(self.count / 100)
        self.create('money', self.curve * self.id)
        
        

In [21]:
simulation = abcEconomics.Simulation(name='gatherdata', processes=1)

It is specified which agents group collects which variables and possessions.

In [22]:
datadealers = simulation.build_agents(DataDealer, 'datadealer', number=10)

Every round the groups need to be instructed to collect the according data. **simulation.finalize()** must be called after the simulation, to write the data! Otherwise the program hangs.

In [23]:

for r in range(100):
    simulation.time = r
    datadealers.counting()
    datadealers.agg_log(variables=['count'])
    datadealers.panel_log(goods=['money'], variables=['curve'])
simulation.finalize()    
        

Round0
Round1
Round2
Round3
Round4
Round5
Round6
Round7
Round8
Round9
Round10
Round11
Round12
Round13
Round14
Round15
Round16
Round17
Round18
Round19
Round20
Round21
Round22
Round23
Round24
Round25
Round26
Round27
Round28
Round29
Round30
Round31
Round32
Round33
Round34
Round35
Round36
Round37
Round38
Round39
Round40
Round41
Round42
Round43
Round44
Round45
Round46
Round47
Round48
Round49
Round50
Round51
Round52
Round53
Round54
Round55
Round56
Round57
Round58
Round59
Round60
Round61
Round62
Round63
Round64
Round65
Round66
Round67
Round68
Round69
Round70
Round71
Round72
Round73
Round74
Round75
Round76
Round77
Round78
Round79
Round80
Round81
Round82
Round83
Round84
Round85
Round86
Round87
Round88
Round89
Round90
Round91
Round92
Round93
Round94
Round95
Round96
Round97
Round98
Round99

time only simulation   1.53
time with data   1.69


We can find the directory of the simulation data by using the `simulation.path` property

In [24]:
print(simulation.path)

/Users/taghawi/Dropbox/workspace/abce_examples/examples/jupyter_tutorial/result/gatherdata_2018-05-11_13-00


In that directory are the data files and a describtion.txt

In [25]:
import os
os.listdir(simulation.path)

['panel_datadealer.csv',
 'aggregated_datadealer.csv',
 'aggregate_datadealer.csv',
 'description.txt']

## Using statistical software

In [26]:
import pandas as pd

In [27]:
%matplotlib inline


ModuleNotFoundError: No module named 'matplotlib'

In [28]:
df = pd.read_csv(simulation.path + '/panel_datadealer.csv')

In [29]:
df.head(20)

,index,curve,money,round,name
0,1,0.010000,0.039999,0,datadealer4
1,2,0.010000,0.049999,0,datadealer5
2,3,0.010000,0.079999,0,datadealer8
3,4,0.010000,0.020000,0,datadealer2
4,5,0.010000,0.089999,0,datadealer9
5,6,0.010000,0.030000,0,datadealer3
6,7,0.010000,0.000000,0,datadealer0
7,8,0.010000,0.010000,0,datadealer1
8,9,0.010000,0.059999,0,datadealer6
9,10,0.010000,0.069999,0,datadealer7


In [32]:
df = df.pivot(index='round', columns='name', values='money')

In [33]:
df.plot()

NameError: name '_converter' is not defined

When running a simulation with python from a start.py simulation.graphs() displays all recorded data. You can also use the @gui decorator to ship abce as an interactive web-app.

# Communication between Simulation and Agents

In [ ]:
simulation = abce.Simulation()

In [ ]:
class Communicator(abce.Agent):
    def report_my_name(self):
        return self.name
    
    def do_as_told(self, time, text):
        print("it is %i o'clock and I have to say %s" % (time, text))

In [ ]:
communicators = simulation.build_agents(Communicator, 'communicator', number=3)

In [ ]:
for time in range(3):
    simulation.advance_round(time)
    communicators.report_my_name()
    communicators.do_as_told(time=time, text="Right says fred")
    
    

One caviat is that **only keyword arguments** work.